In [1]:
import matplotlib.pyplot as plt
import numpy as np 
from numpy import linspace
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing

In [2]:
df = pd.read_csv('data/03_eda_loans50k.csv')

In [3]:
df.head()

,Unnamed: 0,loanID,amount,term,rate,payment,grade,employment,length,home,...,totalAcc,totalBal,totalRevLim,accOpen24,avgBal,bcOpen,bcRatio,totalRevBal,totalBcLim,totalIlLim
0,0,188861,8000.0,36,0.14,272.07,C,Warehouseman,3 years,RENT,...,15.0,48054.0,8100.0,8.0,4369.0,43.0,95.7,48054.0,1000.0,52529.0
1,1,517703,11000.0,36,0.10,354.89,B,Vice President,10+ years,MORTGAGE,...,15.0,265838.0,33400.0,4.0,22153.0,19756.0,37.5,11844.0,31600.0,0.0
2,2,268587,35000.0,36,0.15,1220.33,D,Owner/Attorney,10+ years,RENT,...,14.0,850150.0,83000.0,2.0,121450.0,7686.0,83.2,173260.0,83000.0,100000.0
3,3,579902,20000.0,60,0.12,447.83,C,Analyst,2 years,MORTGAGE,...,48.0,20671.0,14200.0,8.0,1723.0,11061.0,1.2,20671.0,11200.0,32764.0
4,4,617630,12000.0,60,0.12,266.88,B,medical technician,10+ years,MORTGAGE,...,22.0,128034.0,14600.0,3.0,18291.0,5904.0,58.4,8475.0,14200.0,0.0


**Confirm that loanIDs are all unqiue. This number is not important. Using the index is adequate to identify records.**

In [4]:
df.shape

(49184, 31)

In [5]:
df.loanID.nunique()

49184

**Delete duplicated index row, replace existing index with loanID column. Rate, payment & grade are all determined by the bank after approving the loan. These are not predictive features.**

In [6]:
df = df.drop(['Unnamed: 0', 'rate', 'payment', 'grade'], axis=1).set_index('loanID')

**I wanted to see if the employmet field could be used as a predictor. There are over 18,000 unique professions. I also looked to see how many might be duplicates by looking at any employment that contains 'nurse.' There are many that could be grouped but some are pretty different (like registered nurse vs certified nurse assistant). In the end, it's too much to figure out so I dropped that feature. I still have length (employment legth) so I am able to capture some of a related feature still.**

In [7]:
df.employment = df.employment.str.lower()

In [8]:
df.employment.nunique()

18272

In [9]:
search_word = 'nurse'
pattern = f'.*{search_word}.*'  # regular expression pattern to match any string that contains the search_word
result = df[df['employment'].str.contains(pattern)]
result.head(10)

,amount,term,employment,length,home,income,verified,status,reason,state,...,totalAcc,totalBal,totalRevLim,accOpen24,avgBal,bcOpen,bcRatio,totalRevBal,totalBcLim,totalIlLim
loanID,,,,,,,,,,,,,,,,,,,,,
281259,10000.0,36,registered nurse,2 years,RENT,75000.0,Not Verified,Current,debt_consolidation,CA,...,31.0,93495.0,39900.0,2.0,4921.0,31333.00,10.7,93495.0,35100.0,101320.0
54149,18825.0,60,nurse,1 year,MORTGAGE,53000.0,Verified,Current,debt_consolidation,IL,...,12.0,197390.0,13600.0,5.0,28199.0,1868.00,75.4,44885.0,7600.0,44953.0
281495,17000.0,36,registered nurse,7 years,MORTGAGE,140000.0,Not Verified,Current,credit_card,CA,...,32.0,506667.0,47800.0,6.0,31667.0,12753.00,62.4,36045.0,33900.0,30108.0
351772,6000.0,36,certified nurse assistant,2 years,RENT,20000.0,Verified,Default,home_improvement,CA,...,19.0,38782.0,14000.0,8.0,2155.0,9855.00,4.3,38782.0,10300.0,35250.0
616263,8675.0,36,travel nurse,< 1 year,RENT,56400.0,Not Verified,Current,house,SC,...,15.0,29088.0,5800.0,2.0,2909.0,9154.93,63.2,29088.0,0.0,29997.0
214963,25000.0,36,reg nurse,2 years,MORTGAGE,150000.0,Verified,Current,debt_consolidation,MD,...,21.0,463748.0,31800.0,0.0,51528.0,3699.00,87.6,39625.0,29800.0,18006.0
345517,28000.0,36,nurse- aprn,10+ years,MORTGAGE,110000.0,Not Verified,Current,debt_consolidation,SC,...,32.0,134087.0,109200.0,1.0,7449.0,38235.00,48.1,48256.0,73700.0,14986.0
455922,23550.0,36,registered nurse,< 1 year,OWN,56100.0,Verified,Current,debt_consolidation,VA,...,29.0,43342.0,10600.0,3.0,8668.0,342.00,82.9,43342.0,2000.0,44928.0
397210,9000.0,36,public health nurse,1 year,RENT,62000.0,Verified,Current,medical,CO,...,21.0,49724.0,12300.0,2.0,4972.0,2792.00,71.5,49724.0,9800.0,61942.0


In [10]:
df = df.drop('employment', axis=1)

**Feature Encoding using One Hot Encoding on all categorical data. This took the number of features from 26 to 95.**

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49184 entries, 188861 to 500393
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   amount       49184 non-null  float64
 1   term         49184 non-null  int64  
 2   length       49184 non-null  object 
 3   home         49184 non-null  object 
 4   income       49184 non-null  float64
 5   verified     49184 non-null  object 
 6   status       49184 non-null  object 
 7   reason       49184 non-null  object 
 8   state        49184 non-null  object 
 9   debtIncRat   49184 non-null  float64
 10  delinq2yr    49184 non-null  float64
 11  inq6mth      49184 non-null  float64
 12  openAcc      49184 non-null  float64
 13  pubRec       49184 non-null  float64
 14  revolRatio   49184 non-null  float64
 15  totalAcc     49184 non-null  float64
 16  totalBal     49184 non-null  float64
 17  totalRevLim  49184 non-null  float64
 18  accOpen24    49184 non-null  float64
 19

In [12]:
# Select the categorical variables to be encoded
cat_vars = ['term', 'length', 'home', 'verified', 'status', 'reason', 'state']

df_encoded = pd.get_dummies(df, columns=cat_vars, drop_first=True)
df_encoded.columns

Index(['amount', 'income', 'debtIncRat', 'delinq2yr', 'inq6mth', 'openAcc',
       'pubRec', 'revolRatio', 'totalAcc', 'totalBal', 'totalRevLim',
       'accOpen24', 'avgBal', 'bcOpen', 'bcRatio', 'totalRevBal', 'totalBcLim',
       'totalIlLim', 'term_60', 'length_1 year', 'length_10+ years',
       'length_2 years', 'length_3 years', 'length_4 years', 'length_5 years',
       'length_6 years', 'length_7 years', 'length_8 years', 'length_9 years',
       'length_< 1 year', 'home_OWN', 'home_RENT', 'verified_Verified',
       'status_Default', 'reason_credit_card', 'reason_debt_consolidation',
       'reason_home_improvement', 'reason_house', 'reason_major_purchase',
       'reason_medical', 'reason_moving', 'reason_other',
       'reason_renewable_energy', 'reason_small_business', 'reason_vacation',
       'reason_wedding', 'state_AL', 'state_AR', 'state_AZ', 'state_CA',
       'state_CO', 'state_CT', 'state_DC', 'state_DE', 'state_FL', 'state_GA',
       'state_HI', 'state_IL', 'stat

In [13]:
len(df_encoded.columns)

94

In [14]:
df_encoded.dtypes.value_counts()

uint8      76
float64    18
dtype: int64

In [15]:
# Change uint8 data tpes to int
uint8_cols = df_encoded.select_dtypes(include='uint8').columns
df_encoded[uint8_cols] = df_encoded[uint8_cols].astype('int')

In [16]:
df_encoded.dtypes.value_counts()

int64      76
float64    18
dtype: int64

**Separate the features and target variable.**

In [17]:
X = df_encoded.drop('status_Default', axis=1)
y = df_encoded['status_Default']

**Use chi-squared test to limit features.**

In [18]:
# Calculate the chi-squared test statistic and p-value for each feature
chi2, p_values = [], []
for feature in X.columns:
    contingency_table = pd.crosstab(X[feature], y)
    chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
    chi2.append(chi2_stat)
    p_values.append(p_value)

# Select the top k features with the highest chi-squared test statistic
k = 15
top_features = X.columns[np.argsort(chi2)[-k:]]

In [19]:
top_features

Index(['verified_Verified', 'inq6mth', 'term_60', 'accOpen24', 'bcRatio',
       'totalBcLim', 'amount', 'totalRevLim', 'debtIncRat', 'income', 'bcOpen',
       'avgBal', 'totalIlLim', 'totalRevBal', 'totalBal'],
      dtype='object')

In [20]:
X = X[top_features]

**Train / Test Split & Scale using StandardScaler.**

In [21]:
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# initialize a StandardScaler object and fit it on the training data
scaler = StandardScaler()
scaler.fit(X_train)

# apply the scaling transformation to both the training and test data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

**I wanted to check to see if there were a similar amount of defaults in the training and test set. The ratio is close enough.**

In [22]:
ratio_defaults_train = y_train.sum() / len(y_train)
ratio_defaults_test = y_test.sum() / len(y_test)
print(f"The ratio of defaults in the training set is {ratio_defaults_train:.2%} and the ratio in the test set is \
{ratio_defaults_test:.2%}")

The ratio of defaults in the training set is 15.53% and the ratio in the test set is 14.95%


**Data is ready for modeling.**

In [23]:
df_encoded.to_csv('data/04_preprocess_loans50k.csv') 